# RNN

In [9]:
import pandas as pd
import os, time
import random
from IPython.display import clear_output as clear
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import gzip

import csv
from torch.nn.utils.rnn import pack_padded_sequence
from rdkit import Chem
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
import pandas as pd
import os

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

In [2]:
pce = []
SMILES = []
for data_split in os.listdir('./data'):
    data_split = './data/'+data_split
    SMILES += pd.read_csv(data_split)['SMILES_str'].tolist()
    pce += pd.read_csv(data_split)['pce'].tolist()    
SMILES = np.array(SMILES)

rdkit_ok_index = pd.read_csv("./index_rdkit_ok.csv").iloc[:,0].tolist()
random.seed(a=1)
indexs = random.sample(rdkit_ok_index, 10000)
SMILES_sample = [SMILES[index] for index in indexs]
pce_sample = [pce[index] for index in indexs]
index_0 = []
for i in indexs:
    if pce[i]==0:
        index_0.append(i)
        
index_0_91 = random.sample(index_0, 91)

index_ban = []
for i in indexs:
    if i in index_0:
        if i in index_0_91:
            index_ban.append(i)
    else:
        index_ban.append(i)
print(len(index_ban))

SMILES_sample_ban = [SMILES[index] for index in index_ban]
pce_sample_ban = [pce[index] for index in index_ban]

9595


In [18]:
# RNN k_flod 
def data_train_kf(data, hidden_size=200, n_layers=2, n_epoch=100, batch_size=16, USE_GPU = False, set_cv=5):
    def createTensor(tensor):
        if USE_GPU:
            device = torch.device("cuda:0")
            tensor = tensor.to(device)
        return tensor

    class RNNClassifier(torch.nn.Module):
        def __init__(self, input_size, hidden_size, output_size, num_layers = 1, bidirectional = True):
            super(RNNClassifier, self).__init__()
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            self.num_directions = 2 if bidirectional else 1

            self.embedding = torch.nn.Embedding(input_size, hidden_size)
            self.gru = torch.nn.GRU(hidden_size, hidden_size, num_layers, bidirectional = bidirectional)

            self.fc = torch.nn.Linear(hidden_size * self.num_directions, output_size)

        def initHidden(self, batch_size):
            hidden = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_size)
            return createTensor(hidden)

        def forward(self, input, seq_lengths):
            input = input.t()
            batch_size = input.size(1)
            hidden = self.initHidden(batch_size)

            embedding = self.embedding(input)

            gru_input = pack_padded_sequence(embedding, seq_lengths)
            output, hidden = self.gru(gru_input, hidden)
            if self.num_directions == 2:
                hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim = 1)
            else:
                hidden_cat = hidden[-1]

            fc_output = self.fc(hidden_cat)
            return fc_output

    def smiles_to_ASCIIlist(smiles_list):
        ASCIIlist = [ord(smiles) for smiles in smiles_list]
        return ASCIIlist

    def makeTensors(smiles_list, tox_list):
        smiles_sequences = [smiles_to_ASCIIlist(smiles) for smiles in smiles_list]
        smiles_seq_lens = torch.LongTensor([len(name_ASCII) for name_ASCII in smiles_sequences])

        smiles_tensor = torch.zeros(len(smiles_sequences), smiles_seq_lens.max()).long()
        for index, (smiles_sequence, smiles_seq_len) in enumerate(zip(smiles_sequences, smiles_seq_lens), 0):
            smiles_tensor[index, 0:smiles_seq_len] = torch.LongTensor(smiles_sequence)

        ordered_smiles_seq_lens, len_indexes = smiles_seq_lens.sort(dim = 0, descending = True)
        ordered_smiles_tensor = smiles_tensor[len_indexes]
        ordered_tox_list = tox_list[len_indexes]

        return createTensor(ordered_smiles_tensor), createTensor(ordered_smiles_seq_lens), createTensor(ordered_tox_list) 

    def train():
        loss = 0.0
        for batch_index, (smiles, tox) in enumerate(train_loader):

            inputs, seq_lens, targets = makeTensors(smiles, tox)
            if USE_GPU:
                outputs = classifier_model(inputs, seq_lens.cpu())
            else:
                outputs = classifier_model(inputs, seq_lens)
            targets = targets.to(torch.float32).reshape(-1,1)

            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss += loss.item()

            if batch_index % 50 == 49:
                print('time_elapsed: {:.1f}m {:.1f}s, Epoch {}, '.format(timePassed(start_time)[0], timePassed(start_time)[1], epoch), end = '')
                print(f'[{(batch_index+1) * len(inputs)} / {len(training_set)}] ', end = '')
                print(f'loss = {loss / ((batch_index+1) * len(inputs))}')

        return loss/len(train_loader)

    def test():
        correct = 0
        total_samples = len(test_set)
        with torch.no_grad():
            pred = []
            obs = []
            for i, (smiles, tox) in enumerate(test_loader):
                inputs, seq_lens, targets = makeTensors(smiles, tox)
                if USE_GPU:
                    outputs = classifier_model(inputs, seq_lens.cpu())
                    pred.append(outputs.data.cpu())
                    obs.append(targets.data.reshape(-1,1).cpu())
                else:
                    outputs = classifier_model(inputs, seq_lens)
                    pred.append(outputs.data)
                    obs.append(targets.data.reshape(-1,1))
            pred = np.vstack(pred).ravel()
            obs = np.vstack(obs).ravel()
            r2 = r2_score(obs, pred)
            Mse = mse(obs, pred)
            print('r2 on test: %.3f %%' % (100 * r2), 'mse on test: %.5f \n' % (Mse))
        return r2, Mse, pred, obs

    def timePassed(start_time):
        time_passed = time.time() - start_time
        minute = math.floor(time_passed / 60)
        second = time_passed - minute * 60
        return [minute, second]

    os.makedirs("../data_vf_tp/{}/results/DL/1/RNN/test/".format(data), exist_ok=True)
    os.makedirs("../data_vf_tp/{}/results/DL/1/RNN/train/".format(data), exist_ok=True)
    train_smiles = SMILES_sample[:8000]
    train_tox = [float(i) for i in pce_sample[:8000]]
    training_set = [(train_smiles[i], train_tox[i]) for i in range(len(train_smiles))]

    test_smiles = SMILES_sample[8000:10000]
    test_tox = [float(i) for i in pce_sample[8000:10000]]
    test_set = [(test_smiles[i], test_tox[i]) for i in range(len(test_smiles))]

    seed = 1
    base_indices = np.arange(0,len(train_tox))
    np.random.seed(seed)
    np.random.shuffle(base_indices)
    np.random.seed(seed)
    np.random.shuffle(base_indices)
    
    step = int(len(train_tox)/set_cv)
    pred_kflod = pd.DataFrame(index=range(len(train_tox)), columns=["cv1~{}".format(set_cv), "True"])

    for cv in range(set_cv):
        print("*"*20, "Kflod", cv ,"*"*20)
        index = base_indices
        if cv < set_cv-1:
            index_train = np.concatenate([index[:cv*step],index[(cv+1)*step:]], axis=0)
            index_val = index[cv*step:(cv+1)*step]
        else: 
            index_train = index[0:cv*step]
            index_val = index[cv*step:]

        # parameters
        n_chars = 0
        for smiles in train_smiles:
            for char in smiles:
                if n_chars < ord(char):
                    n_chars = ord(char)
        for smiles in test_smiles:
            for char in smiles:
                if n_chars < ord(char):
                    n_chars = ord(char)
        n_chars = n_chars+1
        output_size = 1
        train_loader = DataLoader(dataset = [training_set[index] for index in index_train], batch_size = batch_size, shuffle = True)
        test_loader = DataLoader(dataset = [training_set[index] for index in index_val], batch_size = batch_size, shuffle = False)

        classifier_model = RNNClassifier(n_chars, hidden_size, output_size, n_layers) # input_size, hidden_size, output_size, num_layers
        if USE_GPU:
            device = torch.device("cuda:0")
            classifier_model.to(device)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(classifier_model.parameters(), lr = 0.001, weight_decay=10 ** (-5.0))

        # training and test
        start_time = time.time()
        print("The num of total training epochs is %d. " % n_epoch)
        r2_list = []
        Mse_list = []
        best_r2 = 0
        for epoch in range(n_epoch):
            train()
            r2, Mse, pred, obs = test()
            r2_list.append(r2)
            Mse_list.append(Mse)
            if best_r2 < r2:
                best_r2 = r2
                pred_kflod.iloc[index_val,0] = pred.reshape(1,-1)
                pred_kflod.iloc[index_val,1] = obs.reshape(1,-1)

        plt.figure()
        plt.plot(np.arange(1, n_epoch+1), r2_list)
        plt.plot(np.arange(1, n_epoch+1), Mse_list)
        plt.xlabel("epoch")
        plt.ylabel("r2 & MSE")
        plt.grid()
        plt.show()

    display(pred_kflod)
    pred_kflod.to_csv("../data_vf_tp/{}/results/DL/1/RNN/train/kf_pred_all.csv".format(data), index=0)
    print(r2_score(pred_kflod.iloc[:,1], pred_kflod.iloc[:,0]))

data_train_kf("or_pho_pce", hidden_size=200, n_layers=2, n_epoch=300, batch_size=32, USE_GPU = False, set_cv=5)

In [39]:
# RNN test
def data_train_test(data, hidden_size=200, n_layers=2, n_epoch=100, batch_size=16, USE_GPU = False):
    
    def createTensor(tensor):
        if USE_GPU:
            device = torch.device("cuda:0")
            tensor = tensor.to(device)
        return tensor

    class RNNClassifier(torch.nn.Module):
        def __init__(self, input_size, hidden_size, output_size, num_layers = 1, bidirectional = True):
            super(RNNClassifier, self).__init__()
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            self.num_directions = 2 if bidirectional else 1
            self.embedding = torch.nn.Embedding(input_size, hidden_size)

            self.gru = torch.nn.GRU(hidden_size, hidden_size, num_layers, bidirectional = bidirectional)

            self.fc = torch.nn.Linear(hidden_size * self.num_directions, output_size)

        def initHidden(self, batch_size):
            hidden = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_size)
            return createTensor(hidden)

        def forward(self, input, seq_lengths):
            input = input.t()
            batch_size = input.size(1)
            hidden = self.initHidden(batch_size)

            # embedding layer
            embedding = self.embedding(input)
            # GRU
            gru_input = pack_padded_sequence(embedding, seq_lengths)
            output, hidden = self.gru(gru_input, hidden)
            if self.num_directions == 2:
                hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim = 1)
            else:
                hidden_cat = hidden[-1] 

            fc_output = self.fc(hidden_cat)
            return fc_output

    def smiles_to_ASCIIlist(smiles_list):
        ASCIIlist = [ord(smiles) for smiles in smiles_list]
        return ASCIIlist

    def makeTensors(smiles_list, tox_list):
        smiles_sequences = [smiles_to_ASCIIlist(smiles) for smiles in smiles_list]
        smiles_seq_lens = torch.LongTensor([len(name_ASCII) for name_ASCII in smiles_sequences])

        smiles_tensor = torch.zeros(len(smiles_sequences), smiles_seq_lens.max()).long()
        for index, (smiles_sequence, smiles_seq_len) in enumerate(zip(smiles_sequences, smiles_seq_lens), 0):
            smiles_tensor[index, 0:smiles_seq_len] = torch.LongTensor(smiles_sequence)

        ordered_smiles_seq_lens, len_indexes = smiles_seq_lens.sort(dim = 0, descending = True)
        ordered_smiles_tensor = smiles_tensor[len_indexes]
        ordered_tox_list = tox_list[len_indexes]

        return createTensor(ordered_smiles_tensor), createTensor(ordered_smiles_seq_lens), createTensor(ordered_tox_list) 

    def train():
        loss = 0.0
        for batch_index, (smiles, tox) in enumerate(train_loader): 

            inputs, seq_lens, targets = makeTensors(smiles, tox) 
            if USE_GPU:
                outputs = classifier_model(inputs, seq_lens.cpu())
            else:
                outputs = classifier_model(inputs, seq_lens)
            targets = targets.to(torch.float32).reshape(-1,1)

            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss += loss.item()

        return loss/len(train_loader)

    def test():
        correct = 0
        total_samples = len(test_set)
        with torch.no_grad():
            pred = []
            obs = []
            for i, (smiles, tox) in enumerate(test_loader):
                inputs, seq_lens, targets = makeTensors(smiles, tox)
                if USE_GPU:
                    outputs = classifier_model(inputs, seq_lens.cpu())
                    pred.append(outputs.data.cpu())
                    obs.append(targets.data.reshape(-1,1).cpu())
                else:
                    outputs = classifier_model(inputs, seq_lens)
                    pred.append(outputs.data)
                    obs.append(targets.data.reshape(-1,1))
            pred = np.vstack(pred).ravel()
            obs = np.vstack(obs).ravel()
            r2 = r2_score(obs, pred)
            Mse = mse(obs, pred)
        return r2, Mse, pred, obs

    def timePassed(start_time):
        time_passed = time.time() - start_time
        minute = math.floor(time_passed / 60)
        second = time_passed - minute * 60
        return [minute, second]

    os.makedirs("./results/DL/RNN/test/".format(data), exist_ok=True)
    os.makedirs("./results/DL/RNN/train/".format(data), exist_ok=True)

    train_smiles = SMILES_sample[:8000]
    train_tox = [float(i) for i in pce_sample[:8000]]
    training_set = [(train_smiles[i], train_tox[i]) for i in range(len(train_smiles))]

    test_smiles = SMILES_sample[8000:10000]
    test_tox = [float(i) for i in pce_sample[8000:10000]]
    test_set = [(test_smiles[i], test_tox[i]) for i in range(len(test_smiles))]

    n_chars = 0
    for smiles in train_smiles:
        for char in smiles:
            if n_chars < ord(char):
                n_chars = ord(char)
    for smiles in test_smiles:
        for char in smiles:
            if n_chars < ord(char):
                n_chars = ord(char)
    n_chars = n_chars+1
    output_size = 1
    
    train_loader = DataLoader(dataset = training_set, batch_size = batch_size, shuffle = True)
    test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = False)
    
    classifier_model = RNNClassifier(n_chars, hidden_size, output_size, n_layers) # input_size, hidden_size, output_size, num_layers
    if USE_GPU:
        device = torch.device("cuda:0")
        classifier_model.to(device)
    
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(classifier_model.parameters(), lr = 0.001, weight_decay=10 ** (-5.0))

    start_time = time.time()
    print("The num of total training epochs is %d. " % n_epoch)
    r2_list = []
    Mse_list = []
    best_r2 = 0
    train_loss = []
    for epoch in range(n_epoch):
        train_loss.append(train())
        r2, Mse, pred, obs = test()
        r2_list.append(r2)
        Mse_list.append(Mse)
        if best_r2 < r2:
            best_r2 = r2
            pd.DataFrame([pred, obs], index=["pred", "true"]).T.to_csv('./results/DL/RNN/test/test_pred.csv'.format(data), index=0)
            
        # display
        clear()
        plt.figure(figsize=[16,9])
        plt.plot(np.arange(1, epoch+2), r2_list, label='test_r2')
        plt.plot(np.arange(1, epoch+2), Mse_list, label='test_mse')
        plt.plot(np.arange(1, epoch+2), train_loss, label='train_mse')
        plt.xlabel("epoch")
        plt.ylabel("r2 & MSE")
        plt.legend()
        plt.grid()
        plt.show()
        
        for i in range(epoch+1):
            print('r2 on test: %.3f %%' % (100 * r2_list[i]), 'mse on test: %.5f \n' % (Mse_list[i]), 'mse on train: %.5f \n' % (train_loss[i]))
    
    pd.DataFrame(r2_list).to_csv("./results/DL/RNN/test/scores_r2.csv".format(data), index=0)
    pd.DataFrame(Mse_list).to_csv("./results/DL/RNN/test/scores_mse.csv".format(data), index=0)
    pd.DataFrame(train_loss).to_csv("./results/DL/RNN/test/train_loss.csv".format(data), index=0)
    
    print(r2_list.index(max(r2_list)))
    print(max(r2_list))
    
data_train_test("or_pho_pce", hidden_size=200, n_layers=2, n_epoch=300, batch_size=32, USE_GPU = False)

d:\anaconda3\envs\kekule\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# GNN

In [4]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import os
from rdkit import Chem
from rdkit import RDPaths
from rdkit.Chem import rdmolops, rdmolfiles

import dgl
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from dgl import model_zoo

from dgl.data.chem.utils import mol_to_complete_graph, mol_to_bigraph
# from dgllife.utils import mol_to_bigraph

from dgl.data.chem.utils import atom_type_one_hot
from dgl.data.chem.utils import atom_degree_one_hot
from dgl.data.chem.utils import atom_formal_charge
from dgl.data.chem.utils import atom_num_radical_electrons
from dgl.data.chem.utils import atom_hybridization_one_hot
from dgl.data.chem.utils import atom_total_num_H_one_hot
from dgl.data.chem.utils import one_hot_encoding
from dgl.data.chem import CanonicalAtomFeaturizer
from dgl.data.chem import CanonicalBondFeaturizer
from dgl.data.chem import ConcatFeaturizer
from dgl.data.chem import BaseAtomFeaturizer
from dgl.data.chem import BaseBondFeaturizer

from dgl.data.chem import one_hot_encoding
from dgl.data.utils import split_dataset


# from dgllife.utils import one_hot_encoding, atom_type_one_hot, atom_formal_charge, atom_num_radical_electrons 

from functools import partial
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

import pandas as pd
from IPython.display import display, clear_output

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [40]:
# GNN k_fold
set_cv = 5
n_epochs = 200

def chirality(atom):
    try:
        return one_hot_encoding(atom.GetProp('_CIPCode'), ['R', 'S']) + \
               [atom.HasProp('_ChiralityPossible')]
    except:
        return [False, False] + [atom.HasProp('_ChiralityPossible')]


def collate_molgraphs(data):
    assert len(data[0]) in [3, 4], \
        'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
    if len(data[0]) == 3:
        smiles, graphs, labels = map(list, zip(*data))
        masks = None
    else:
        smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)
    return smiles, bg, labels, masks


def run_a_train_epoch(n_epochs, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    total_loss = 0
    losses = []
    pred = []
    best_r2 = 0
    for batch_id, batch_data in enumerate(data_loader):
        batch_data
        smiles, bg, labels, masks = batch_data
        bg.to(torch.device('cpu'))
        labels = labels.to('cpu')
        masks = masks.to('cpu')

        prediction = model(bg, bg.ndata['hv'], bg.edata['he'])
        pred.append(prediction.detach().numpy())
        loss = (loss_criterion(prediction, labels) * (masks != 0).float()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.data.item())

    model.eval()
    all_pred = []
    all_y = []
    for test_data in test_loader:
        smi_lst, bg, labels, masks = test_data
        bg.to(torch.device('cpu'))
        labels = labels.to('cpu')
        masks = masks.to('cpu')
        pred = model(bg, bg.ndata['hv'], bg.edata['he'])
        all_pred.append(pred.data.cpu().numpy())
        all_y.append(labels)

    res = np.vstack(all_pred)
    r2 = r2_score(np.vstack(all_y), np.vstack(res))
    if r2 > best_r2:
        best_r2 = r2
        pred_kflod.iloc[index_val, 0] = np.ravel(res)
        pred_kflod.iloc[index_val, 1] = np.ravel(np.vstack(all_y))

    Mse = mse(np.vstack(all_y), np.vstack(res))
    r2_all.append(r2)
    mse_all.append(Mse)
    total_score = np.mean(losses)
    print('epoch {:d}/{:d}, cv: {} train mse: {:.4f} test r_2: {:.4f} mse: {:.4f}'.format(epoch + 1, n_epochs, cv, total_score, r2, Mse), end="")
    return total_score

atom_featurizer = BaseAtomFeaturizer(
                 {'hv': ConcatFeaturizer([
                  partial(atom_type_one_hot, allowable_set=[
                          'B', 'C', 'N', 'O', 'F', 'Si', 'P', 'S', 'Cl', 'As', 'Se', 'Br', 'Te', 'I', 'At'],
                    encode_unknown=True),
                  partial(atom_degree_one_hot, allowable_set=list(range(6))),
                  atom_formal_charge, atom_num_radical_electrons,
                  partial(atom_hybridization_one_hot, encode_unknown=True),
                  lambda atom: [0], # A placeholder for aromatic information,
                    atom_total_num_H_one_hot, chirality
                 ],
                )})
bond_featurizer = BaseBondFeaturizer({
                                     'he': lambda bond: [0 for _ in range(10)]
    })

os.makedirs("./results/DL/GNN/test/", exist_ok=True)
os.makedirs("./results/DL/GNN/train/", exist_ok=True)

train_smi = SMILES_sample[:8000]
train_mols = [Chem.MolFromSmiles(smiles) for smiles in train_smi]
train_sol = torch.tensor(pce_sample[:8000]).reshape(-1, 1)

test_smi = SMILES_sample[8000:10000]
test_mols = [Chem.MolFromSmiles(smiles) for smiles in test_smi]
test_sol = torch.tensor(pce_sample[8000:10000]).reshape(-1, 1)

train_graph = [mol_to_bigraph(mol,
                              node_featurizer=atom_featurizer,
                              edge_featurizer=bond_featurizer) for mol in train_mols]

test_graph = [mol_to_bigraph(mol,
                             node_featurizer=atom_featurizer,
                             edge_featurizer=bond_featurizer) for mol in test_mols]

model = model_zoo.chem.AttentiveFP(node_feat_size=39,
                                   edge_feat_size=10,
                                   num_layers=2,
                                   num_timesteps=2,
                                   graph_feat_size=200,
                                   output_size=1,
                                   dropout=0.2)
model = model.to('cpu')

seed = 1
base_indices = np.arange(0,len(train_sol))
np.random.seed(seed)
np.random.shuffle(base_indices)
np.random.seed(seed)
np.random.shuffle(base_indices)
step = int(len(train_sol)/set_cv)
pred_kflod = pd.DataFrame(index=range(len(train_sol)), columns=["cv1~5", "True"])

clear_output()

for cv in range(set_cv):
    print("*"*20, "Kflod", cv ,"*"*20)
    index = base_indices
    if cv < set_cv-1:
        index_train = np.concatenate([index[:cv*step],index[(cv+1)*step:]], axis=0)
        index_val = index[cv*step:(cv+1)*step]
    else: 
        index_train = index[0:cv*step]
        index_val = index[cv*step:]

    train_loader = DataLoader(dataset=list(zip(np.array(train_smi)[index_train], np.array(train_graph)[index_train], train_sol[index_train])), batch_size=122,
                              collate_fn=collate_molgraphs)
    test_loader = DataLoader(dataset=list(zip(np.array(train_smi)[index_val], np.array(train_graph)[index_val], train_sol[index_val])), batch_size=122,
                             collate_fn=collate_molgraphs)

    model = model_zoo.chem.AttentiveFP(node_feat_size=39,
                                   edge_feat_size=10,
                                   num_layers=2,
                                   num_timesteps=2,
                                   graph_feat_size=200,
                                   output_size=1,
                                   dropout=0.2)
    model = model.to('cpu')
    loss_fn = nn.MSELoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr=10 ** (-2.5), weight_decay=10 ** (-5.0),)

    epochs = []
    scores = []
    r2_all = []
    mse_all = []
    for e in range(n_epochs):
        score = run_a_train_epoch(n_epochs, e, model, train_loader, loss_fn, optimizer)
        epochs.append(e)
        scores.append(score)

    plt.plot(epochs, scores)
    plt.savefig(f'./results/{cv}cv_losses.png')
    pd.DataFrame(scores).to_csv(f'./results/{cv}cv_losses.csv')

print(r2_score(pred_kflod.iloc[:,0], pred_kflod.iloc[:,1]))
pred_kflod.to_csv("./results/DL/GNN/train/kf_pred_all.csv", index=0)

d:\anaconda3\envs\kekule\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# GNN test
def data_train_test(data, n_epochs = 100):
    warnings.filterwarnings('ignore')
    def chirality(atom):
        try:
            return one_hot_encoding(atom.GetProp('_CIPCode'), ['R', 'S']) + \
                   [atom.HasProp('_ChiralityPossible')]
        except:
            return [False, False] + [atom.HasProp('_ChiralityPossible')]


    def collate_molgraphs(data):
        assert len(data[0]) in [3, 4], \
            'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
        if len(data[0]) == 3:
            smiles, graphs, labels = map(list, zip(*data))
            masks = None
        else:
            smiles, graphs, labels, masks = map(list, zip(*data))

        bg = dgl.batch(graphs)
        bg.set_n_initializer(dgl.init.zero_initializer)
        bg.set_e_initializer(dgl.init.zero_initializer)
        labels = torch.stack(labels, dim=0)

        if masks is None:
            masks = torch.ones(labels.shape)
        else:
            masks = torch.stack(masks, dim=0)
        return smiles, bg, labels, masks


    def run_a_train_epoch(n_epochs, epoch, model, data_loader, loss_criterion, optimizer):
        model.train()
        total_loss = 0
        losses = []
        pred = []
        best_r2 = 0
        for batch_id, batch_data in enumerate(data_loader):
            batch_data
            smiles, bg, labels, masks = batch_data
            bg.to(torch.device('cpu'))
            labels = labels.to('cpu')
            masks = masks.to('cpu')

            prediction = model(bg, bg.ndata['hv'], bg.edata['he'])
            pred.append(prediction.detach().numpy())
            loss = (loss_criterion(prediction, labels) * (masks != 0).float()).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses.append(loss.data.item())

        model.eval()
        all_pred = []
        for test_data in test_loader:
            smi_lst, bg, labels, masks = test_data
            bg.to(torch.device('cpu'))
            labels = labels.to('cpu')
            masks = masks.to('cpu')
            pred = model(bg, bg.ndata['hv'], bg.edata['he'])
            all_pred.append(pred.data.cpu().numpy())

        res = np.vstack(all_pred)
        r2 = r2_score(test_sol, np.vstack(res))
        Mse = MSE(test_sol, np.vstack(res))
        Mae = MAE(test_sol, np.vstack(res))
        r2_all.append(r2)
        mse_all.append(Mse)
        mae_all.append(Mae)
        if best_r2 < r2:
            best_r2 = r2
            print()
#             print(res)
            pd.DataFrame([res.ravel(), np.array(test_sol).ravel()]).T.to_csv("./results/DL/GNN/test/test_pred_{}.csv".format(epoch), index=0)
        total_score = np.mean(losses)
        print('epoch {:d}/{:d}, training mse: {:.4f} test r_2: {:.4f} mse: {:.4f}'.format(epoch + 1, n_epochs, total_score, r2, Mse), end="")

        return total_score


    atom_featurizer = BaseAtomFeaturizer(
                     {'hv': ConcatFeaturizer([
                      partial(atom_type_one_hot, allowable_set=[
                             'B', 'C', 'N', 'O', 'F', 'Si', 'P', 'S', 'Cl', 'As', 'Se', 'Br', 'Te', 'I', 'Si'],
                        encode_unknown=True),
                      partial(atom_degree_one_hot, allowable_set=list(range(6))),
                      atom_formal_charge, atom_num_radical_electrons,
                      partial(atom_hybridization_one_hot, encode_unknown=True),
                      lambda atom: [0], # A placeholder for aromatic information,
                        atom_total_num_H_one_hot, chirality
                     ],
                    )})
    bond_featurizer = BaseBondFeaturizer({
                                         'he': lambda bond: [0 for _ in range(10)]
        })

    os.makedirs("./results/DL/GNN/test/".format(data), exist_ok=True)
    os.makedirs("./results/DL/GNN/train/".format(data), exist_ok=True)

    train_smi = SMILES_sample[:8000]
    train_mols = [Chem.MolFromSmiles(smiles) for smiles in train_smi]
    train_sol = torch.tensor(pce_sample[:8000]).reshape(-1, 1)

    test_smi = SMILES_sample[8000:10000]
    test_mols = [Chem.MolFromSmiles(smiles) for smiles in test_smi]
    test_sol = torch.tensor(pce_sample[8000:10000]).reshape(-1, 1)

    train_graph = [mol_to_bigraph(mol,
                                  node_featurizer=atom_featurizer,
                                  edge_featurizer=bond_featurizer) for mol in train_mols]

    test_graph = [mol_to_bigraph(mol,
                                 node_featurizer=atom_featurizer,
                                 edge_featurizer=bond_featurizer) for mol in test_mols]

    model = model_zoo.chem.AttentiveFP(node_feat_size=39,
                                       edge_feat_size=10,
                                       num_layers=2,
                                       num_timesteps=2,
                                       graph_feat_size=200,
                                       output_size=1,
                                       dropout=0.2)
    model = model.to('cpu')

    train_loader = DataLoader(dataset=list(zip(train_smi, train_graph, train_sol)), batch_size=16,
                              collate_fn=collate_molgraphs)
    test_loader = DataLoader(dataset=list(zip(test_smi, test_graph, test_sol)), batch_size=16,
                             collate_fn=collate_molgraphs)

    loss_fn = nn.MSELoss(reduction='none')
    optimizer = torch.optim.SGD(model.parameters(), lr=10 ** (-2.5), weight_decay=10 ** (-5.0),)#Adam
    epochs = []
    scores = []
    r2_all = []
    mse_all = []
    mae_all = []
    for e in range(n_epochs):
        score = run_a_train_epoch(n_epochs, e, model, train_loader, loss_fn, optimizer)
        epochs.append(e)
        scores.append(score)

    plt.plot(epochs, scores)
    plt.savefig('111.png')

    pd.DataFrame(r2_all).to_csv("./results/DL/GNN/test/r2_all.csv".format(data), index=0)
    pd.DataFrame(mse_all).to_csv("./results/DL/GNN/test/mse_all.csv".format(data), index=0)
    pd.DataFrame(mae_all).to_csv("./results/DL/GNN/test/mae_all.csv".format(data), index=0)
    pd.DataFrame(scores).to_csv("./results/DL/GNN/test/scores.csv".format(data), index=0)
    print(r2_score(test_sol, res))

    plt.figure()
    plt.plot(range(len(r2_all)), r2_all)
    plt.plot(range(len(r2_all)), mse_all)
    plt.show
    print(max(r2_all), min(mse_all))
    print(r2_all.index(max(r2_all)), mse_all.index(min(mse_all)))
    
    return r2_all, mse_all, mae_all

r2_linshi, mse_linshi, mae_linshi = data_train_test("or_pho_pce", n_epochs = 200)

# CNN

In [41]:
from __future__ import division, print_function
import torch
from argparse import ArgumentParser
import torch

from sklearn.metrics import mean_squared_error
from math import sqrt
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models
import os, glob, time
import copy
import sys
import numpy as np
import scipy
from scipy import stats
from scipy import spatial
import os,sys, os.path
from collections import defaultdict
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit.rdBase
from rdkit import DataStructs
from rdkit.DataStructs import BitVectToText
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG
import IPython

from IPython.core.display import SVG
from torch.autograd import Variable
import multiprocessing

import subprocess
from torchvision import transforms
import pandas as pd
from rdkit.Chem import Draw
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from IPython import display
import random

import torch.utils.data as data
from PIL import Image
import os
import os.path

nets = ["vgg19_bn", "densenet201"]
tox = ["pce"]
argparser = ArgumentParser()
args_group = argparser.add_argument_group(title='Running args')
args_group.add_argument('-seed', type=int,  required=False, default=1)
args_group.add_argument('-architecture', required=False, choices=nets, default="vgg19_bn")
args_group.add_argument('-lr', type=float, required=False, default=0.01)
args_group.add_argument('-batch_size', type=int, required=False, default=16)
args_group.add_argument('-data_augmentation', required=False, default=1,choices=[0,1] )
args_group.add_argument('-nb_epochs_training_per_cycle',  required=False, default=200)
args_group.add_argument('-nb_epochs_training', type=int,  required=False, default=300)
args_group.add_argument('-load_dict', type=list,  required=False, nargs='+', default=[])
args_group.add_argument('-Tox', type=str, required=False, default=0)
args_group.add_argument('-cv', type=int, required=False, default=5)
args_group.add_argument('-step_size_lr_decay', type=int, required=False, default=25)
args_group.add_argument('-drop_factor_lr', type=float, required=False, default=0.6 )
args = argparser.parse_args(['-batch_size', '32'
                             , "-architecture"
                             , "densenet201"
                             ,'-load_dict','0','0','0','0','0'
                             ,'-nb_epochs_training', '200'
                             ,'-lr', '0.01'
                             ,'-nb_epochs_training_per_cycle', '100'
                             ,'-Tox', 'pce'
                             ,'-cv', '5'])

seed = args.seed
net=args.architecture
lr=args.lr

def default_loader(path):
    return Image.open(path).convert('RGB')

def default_flist_reader(flist):
    imlist = []
    with open(flist, 'r') as rf:
        for line in rf.readlines():
            impath, imlabel = line.strip().split()
            imlist.append( (impath, int(imlabel)) )
    
    return imlist

class ImageFilelist(data.Dataset):
    def __init__(self,  paths_labels, transform=None, target_transform=None,
        flist_reader=default_flist_reader, loader=default_loader):

        self.imlist = paths_labels
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        
    def __getitem__(self, index):
        impath, target = self.imlist[index]
        img = self.loader(impath)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
   
        return img, target
        
    def __len__(self):
        return len(self.imlist)

data = pd.read_csv("./moldata.csv")
SMILES = np.array(data.iloc[:,1].to_list())
pce = data.iloc[:,4].to_list()
del data

mols_train = [Chem.MolFromSmiles(smiles) for smiles in SMILES_sample[:8000]]
mols_test = [Chem.MolFromSmiles(smiles) for smiles in SMILES_sample[8000:]]

del SMILES, pce

my_smiles_train = SMILES_sample[:8000]
my_smiles_test = SMILES_sample[8000:]
chembl_ids_train = indexs[:8000]
chembl_ids_test = indexs[8000:]
activities_train = pce_sample[:8000]
activities_test = pce_sample[8000:]

if(len(my_smiles_train)+len(my_smiles_test) != len(activities_train)+len(activities_test)):
    raise "The number of compounds does not correspond to the number of bioactivities"

base_indices = np.arange(0,len(activities_train))
np.random.seed(seed)
np.random.shuffle(base_indices)
np.random.seed(seed)
np.random.shuffle(base_indices)

os.makedirs("./images", exist_ok=True)
os.makedirs("./images/train", exist_ok=True)
os.makedirs("./images/test", exist_ok=True)

svgs = glob.glob( "./images/train/*.svg")
pngs = glob.glob( "./images/train/*.png")
if len(svgs) == 0 and len(pngs) == 0:
    for i,mm in enumerate(mols_train):
        mol_now=[mm]
        koko=Chem.Draw.MolsToGridImage([x for x in mol_now], molsPerRow=1,useSVG=True)
        orig_stdout = sys.stdout
        f = open("./images/train/{}.svg".format(chembl_ids_train[i]), 'w')
        sys.stdout = f
        print(koko.data)
        sys.stdout = orig_stdout
        f.close()
else:
    print("Train SVGs ready")

svgs = glob.glob( "./images/test/*.svg")
pngs = glob.glob( "./images/test/*.png")
if len(svgs) == 0 and len(pngs) == 0:
    for i,mm in enumerate(mols_test):
        mol_now=[mm]
        koko=Chem.Draw.MolsToGridImage([x for x in mol_now], molsPerRow=1,useSVG=True)
        orig_stdout = sys.stdout
        f = open("./images/test/{}.svg".format(chembl_ids_test[i]), 'w')
        sys.stdout = f
        print(koko.data)
        sys.stdout = orig_stdout
        f.close()
else:
    print("Test SVGs ready")

pngs = glob.glob( "./images/train/*.png")
if len(pngs) == 0:
    basedir=os.getcwd()
    os.chdir(("./images/train"))
    cmd = "magick mogrify -sample 224x224 -density 800 -format png *.svg"
    os.system(cmd)
    os.chdir(basedir)
else:
    print("Train PNGs ready")
            
pngs = glob.glob( "./images/test/*.png")
if len(pngs) == 0:
    basedir=os.getcwd()
    os.chdir(("./images/test"))
    cmd = "magick mogrify -sample 224x224 -density 800 -format png *.svg"
    os.system(cmd)
    os.chdir(basedir)
else:
    print("Test PNGs ready")

del svgs
del pngs

if args.data_augmentation == 1:
    transforms = {
            'train': transforms.Compose([
                transforms.Resize(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(degrees=90),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            'val': transforms.Compose([
                transforms.Resize(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(degrees=90),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            'test': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            }
else:
    transforms = {
            'train': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            'val': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            'test': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
            }

paths_labels_train=[]
for i,x in enumerate(activities_train):
    path_now = './images/train/{}.png'.format(chembl_ids_train[i])
    now = (path_now , x)
    paths_labels_train.append(now)

paths_labels_test=[]
for i,x in enumerate(activities_test):
    path_now = './images/test/{}.png'.format(chembl_ids_test[i])
    now = (path_now , x)
    paths_labels_test.append(now)                                               
     
workers=multiprocessing.cpu_count()
workers = 0
shuffle=False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    print("-"*20)
    print("Strat Training {} {} cv: {}".format(args.cell_line, args.Tox, cv+1))
    print(net)
    print("data_augmentation: {}".format(args.data_augmentation))
    print("batch_size: {}".format(args.batch_size))
    start_epoch = -1
    best_epoch = 0
    load_epoch = int(args.load_dict[cv][0])
    if  load_epoch > 0:
        checkpoint = torch.load("./models/checkpoint/{}/{}/{}/{}_{}.pth".format(args.cell_line,args.seed,net,load_epoch, cv))
        print("strat from",checkpoint["epoch"],"epoch")
        
        best_epoch = load_epoch
        optimizer.load_state_dict(checkpoint['optimizer'])
        start_epoch = checkpoint['epoch']
        scheduler.load_state_dict(checkpoint['lr_schedule'])
        net_dict = copy.deepcopy(checkpoint['net'])
        model.load_state_dict(net_dict)
        del net_dict,checkpoint
        
    print("-"*20)
    model.cuda()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_mse = 1000.0
    best_r2 = -10000
    early = 0
    start_time = time.time()
    for epoch in range(start_epoch+1, num_epochs):
        time_epoch = time.time()

        # cyclical learning rate
        if early % args.nb_epochs_training_per_cycle == 0:
            optimizer = optim.SGD(model.parameters(), lr=args.lr)
            scheduler = lr_scheduler.StepLR(optimizer, step_size=args.step_size_lr_decay, gamma=args.drop_factor_lr)
        
        print("-"*20)
        print('Epoch {}/{} early:{}'.format(epoch, num_epochs - 1, early))

        for phase in ['train', 'val']:
            epoch_losses=0.0
            deno=0.0
            if phase == 'train':
                model.train()  
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    labels = labels.type(torch.FloatTensor)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        preds=outputs.squeeze(1)
                        preds = preds.type(torch.FloatTensor)
                        loss = criterion(preds, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    del inputs, outputs, labels
                    epoch_losses += loss.item() * len(preds)
                    deno +=len(preds)
                    del preds
                    
                epoch_loss = epoch_losses / deno
                train_loss.append(epoch_loss)
                print('{} Loss: {:.4f} {}'.format(phase, epoch_loss, deno))
                
            if phase == 'val':
                model.eval()
                pred=[]
                obs=[]
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    labels = labels.type(torch.FloatTensor)

                    optimizer.zero_grad()
            
                    outputs = model(inputs)
                    for i in range(len(labels)):
                        pred.append(float(outputs.data[i]))
                        obs.append(float(labels.data[i]))
                
                    del inputs, outputs, labels
              
                mse = mean_squared_error(obs, pred)
                r2 = r2_score(obs, pred)
                lr = optimizer.state_dict()['param_groups'][0]['lr']
                print('val Loss: {:.4f} r^2: {:.4f} lr: {:.4f}'.format(mse, r2, lr))
                lr_decay.append(lr)
                scores_mse.append(mse)
                scores_r2.append(r2)

            if phase == 'val' and r2 > best_r2:
                best_r2 = r2
                best_mse = mse
                best_model_wts = copy.deepcopy(model.state_dict())
                early=0
                
                kf_pred_all.iloc[index_val, 0] = pred
                kf_pred_all.iloc[index_val, 1] = obs
                
                best_epoch = epoch
                
        
            if phase == 'val' and r2 < best_r2:
                early+=1
                
            if phase == 'train' and early>20:
                scheduler.step()

        print('Epoch complete in {:.0f}m {:.0f}s'.format( (time.time() - time_epoch) // 60, (time.time() - time_epoch) % 60))

    plt.plot(train_loss)
    plt.show()
    plt.plot(scores_r2)
    plt.plot(scores_mse)
    plt.show()
    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best r2: {:4f}'.format(best_r2), 'Best mse: {:4f}'.format(best_mse))
    print("-"*20)
    print("end")
    print("-"*20)
    model.load_state_dict(best_model_wts)
    return model

def train_test(model, criterion, optimizer, scheduler, num_epochs=25):

    print("-"*20)
    print("Strat Training Test  {}".format(args.Tox))
    print(net)
    print("data_augmentation: {}".format(args.data_augmentation))
    print("batch_size: {}".format(args.batch_size))
    start_epoch = -1
    best_epoch = 0
    print("-"*20)
    model.cuda()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_mse = 1000.0
    best_r2 = -10000
    early = 0
    start_time = time.time()
    for epoch in range(start_epoch+1, num_epochs):
        time_epoch = time.time()

        # cyclical learning rate
        if early % args.nb_epochs_training_per_cycle == 0:
            optimizer = optim.SGD(model.parameters(), lr=args.lr)
            scheduler = lr_scheduler.StepLR(optimizer, step_size=args.step_size_lr_decay, gamma=args.drop_factor_lr)
        
        print("-"*20)
        print('Epoch {}/{} early:{}'.format(epoch, num_epochs - 1, early))

        for phase in ['train', 'val']:
            epoch_losses=0.0
            deno=0.0
            if phase == 'train':
                model.train()  
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    labels = labels.type(torch.FloatTensor)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        preds=outputs.squeeze(1)
                        preds = preds.type(torch.FloatTensor)
                        loss = criterion(preds, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    del inputs, outputs, labels
                    epoch_losses += loss.item() * len(preds)
                    deno +=len(preds)
                    del preds
                    
                epoch_loss = epoch_losses / deno
                train_loss.append(epoch_loss)
                print('{} Loss: {:.4f} {}'.format(phase, epoch_loss, deno))
                
            if phase == 'val':
                model.eval()
                pred=[]
                obs=[]
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    labels = labels.type(torch.FloatTensor)
                    optimizer.zero_grad()
            
                    outputs = model(inputs)
                    for i in range(len(labels)):
                        pred.append(float(outputs.data[i]))
                        obs.append(float(labels.data[i]))
                
                    del inputs, outputs, labels

                mse = mean_squared_error(obs, pred)
                r2 = r2_score(obs, pred)
                lr = optimizer.state_dict()['param_groups'][0]['lr']
                print('val Loss: {:.4f} r^2: {:.4f} lr: {:.4f}'.format(mse, r2, lr))
                lr_decay.append(lr)
                scores_mse.append(mse)
                scores_r2.append(r2)

            if phase == 'val' and r2 > best_r2:
                best_r2 = r2
                best_mse = mse
                best_model_wts = copy.deepcopy(model.state_dict())
                early=0
                
                test_pred.iloc[:, 0] = pred
                test_pred.iloc[:, 1] = obs
                test_pred.iloc[0, 2] = best_r2
                test_pred.iloc[0, 3] = best_mse
        
            if phase == 'val' and r2 < best_r2:
                early+=1            
                
            if phase == 'train' and early>20:
                scheduler.step()

        print('Epoch complete in {:.0f}m {:.0f}s'.format( (time.time() - time_epoch) // 60, (time.time() - time_epoch) % 60))
        
        clear_out()
        plt.plot(train_loss)
        plt.show()
        plt.plot(scores_mse)
        plt.show()
    time_elapsed = time.time() - start_time
    print('Training complete in { :.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best r2: {:4f}'.format(best_r2), 'Best mse: {:4f}'.format(best_mse))
    print("-"*20)
    print("end")
    print("-"*20)
    model.load_state_dict(best_model_wts)
    return model

def modelselect(net):
    pretrained = False
    if net == "densenet201": 
        model_sel = models.densenet201(pretrained=pretrained)
        modules=[]
        modules.append( nn.Linear(in_features=1920, out_features=4096, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=4096, out_features=1000, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=1000, out_features=200, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=200, out_features=100, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=100, out_features=1, bias=True) )
        classi = nn.Sequential(*modules)
        model_sel.classifier = classi

    if net == "vgg19_bn": 
        model_sel = models.vgg19_bn(pretrained=pretrained)
        modules=[]
        modules.append( nn.Linear(in_features=25088, out_features=4096, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=4096, out_features=1000, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=1000, out_features=200, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=200, out_features=100, bias=True) )
        modules.append( nn.ReLU(inplace=True) )
        modules.append( nn.Dropout(p=0.5) )
        modules.append( nn.Linear(in_features=100, out_features=1, bias=True) )
        classi = nn.Sequential(*modules)
        model_ft.classifier = classi
    return model_sel

k_folf = False
for net in ["densenet201"]:
    r2_all = []
    lr_all = []
    mse_all = []
    kf_pred_all = pd.DataFrame(index=range(len(mols_train)), columns=["cv1~5", "True"])
    test_score_all = pd.DataFrame(index=list(range(len(mols_test)))+["r2","MSE"], columns=["cv1", "cv2", "cv3", "cv4", "cv5", "True"])
    train_loss_all = []

    if False:
        index = base_indices
        step = int(len(index)/args.cv)
        for cv in range(args.cv):
            model_ft = modelselect(net)
            optimizer_ft = optim.SGD(model_ft.parameters(), lr=args.lr)
            scores_r2 = []
            lr_decay = []
            scores_mse = []
            train_loss = []
            if cv < args.cv-1:
                index_train = np.concatenate([index[:cv*step],index[(cv+1)*step:]], axis=0)
                index_val = index[cv*step:(cv+1)*step]
            else: 
                index_train = index[0:cv*step]
                index_val = index[cv*step:]

            paths_labels_train_train = []
            for i in index_train:
                paths_labels_train_train.append(paths_labels_train[i])

            paths_labels_train_val = []
            for i in index_val:
                paths_labels_train_val.append(paths_labels_train[i])
                
            trainloader = torch.utils.data.DataLoader(
                        ImageFilelist(paths_labels= paths_labels_train_train,
                        transform=transforms['train']),
                        batch_size=args.batch_size, shuffle=shuffle,
                        num_workers=workers) 

            valloader = torch.utils.data.DataLoader(
                        ImageFilelist(paths_labels= paths_labels_train_val,
                        transform=transforms['val']),
                        batch_size=args.batch_size, shuffle=shuffle,
                        num_workers=workers) 

            testloader = torch.utils.data.DataLoader(
                        ImageFilelist(paths_labels= paths_labels_test,
                        transform=transforms['test']),
                        batch_size=args.batch_size, shuffle=shuffle,
                        num_workers=workers) 

            dataloaders = {'train': trainloader, 'val':valloader, 'test':testloader}
            criterion = torch.nn.MSELoss()
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=args.step_size_lr_decay, gamma=args.drop_factor_lr)
            model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=args.nb_epochs_training)

            pred = []
            obs = []
            for inputs, labels in dataloaders['test']:
                inputs = inputs.cuda()
                labels = labels.cuda()
                model_ft.cuda()
                labels = labels.type(torch.FloatTensor)
                outputs = model_ft(inputs)
                for i in range(len(labels)):
                    pred.append(float(outputs.data[i]))
                    obs.append(float(labels.data[i]))

                del inputs, outputs, labels
            mse = mean_squared_error(obs, pred)
            r2 = r2_score(obs, pred)
            test_score_all.iloc[:len(pred),cv] = pred
            test_score_all.iloc[:len(obs),args.cv] = obs
            test_score_all.iloc[len(pred),cv] = r2
            test_score_all.iloc[len(pred)+1,cv] = mse
            print('test Loss: {:.4f} r^2: {}'.format(mse, r2))


            r2_all.append(scores_r2)
            lr_all.append(lr_decay)
            mse_all.append(scores_mse)
            train_loss_all.append(train_loss)
        
        os.makedirs("./results/DL/{}/{}/train".format(args.seed,net), exist_ok=True)
        os.makedirs("./results/DL/{}/{}/test".format(args.seed,net), exist_ok=True)
        pd.DataFrame(r2_all).T.to_csv("./results/DL/{}/{}/train/r2_all.csv".format(args.seed,net),index=None)
        pd.DataFrame(lr_all).T.to_csv("./results/DL/{}/{}/train/lr_all.csv".format(args.seed,net),index=None)
        pd.DataFrame(mse_all).T.to_csv("./results/DL/{}/{}/train/mse_all.csv".format(args.seed,net),index=None)
        pd.DataFrame(kf_pred_all).to_csv("./results/DL/{}/{}/train/kf_pred_all.csv".format(args.seed,net),index=None)
        pd.DataFrame(train_loss_all).T.to_csv("./results/DL/{}/{}/train/train_loss_all.csv".format(args.seed,net),index=None)
        pd.DataFrame(test_score_all).to_csv("./results/DL/{}/{}/test/test_score_all.csv".format(args.seed,net),index=None)
        
    scores_r2 = []
    lr_decay = []
    scores_mse = []
    train_loss = []
    test_pred = pd.DataFrame(index=range(len(activities_test)), columns=["pred", "True", "r2", "mse"])

    trainloader = torch.utils.data.DataLoader(
                ImageFilelist(paths_labels= paths_labels_train,
                transform=transforms['train']),
                batch_size=args.batch_size, shuffle=shuffle,
                num_workers=workers) 

    valloader = torch.utils.data.DataLoader(
                ImageFilelist(paths_labels= paths_labels_test,
                transform=transforms['val']),
                batch_size=args.batch_size, shuffle=shuffle,
                num_workers=workers) 

    dataloaders = {'train': trainloader, 'val':valloader}

    model_ft = modelselect(net)
    criterion = torch.nn.MSELoss()
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=args.lr)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=args.step_size_lr_decay, gamma=args.drop_factor_lr)
    model_ft = train_test(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=args.nb_epochs_training)

    pd.DataFrame(scores_r2).to_csv("./results/DL/{}/{}/test/scores_r2.csv".format(args.seed,net),index=None)
    pd.DataFrame(lr_decay).to_csv("./results/DL/{}/{}/test/lr_decay.csv".format(args.seed,net),index=None)
    pd.DataFrame(scores_mse).to_csv("./results/DL/{}/{}/test/scores_mse.csv".format(args.seed,net),index=None)
    pd.DataFrame(train_loss).to_csv("./results/DL/{}/{}/test/train_loss.csv".format(args.seed,net),index=None)
    pd.DataFrame(test_pred).to_csv("./results/DL/{}/{}/test/test_pred.csv".format(args.seed,net),index=None)

d:\anaconda3\envs\kekule\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
